In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import json
import csv
import re
import os
import xml.etree.ElementTree as et
from urllib.request import urlopen
import datetime
import time
from tqdm import tqdm
import sqlite3

In [ ]:
filename = "..\data\podcast_ID_list"

In [ ]:
podcasts = pd.read_csv(filename, index_col = False)

In [ ]:
# Create connection
con = sqlite3.connect('../data/GuidePod_clean.sqlite')

# import
feeds = pd.read_sql_query("SELECT feedurl from podcast_master;", con)

In [ ]:
def get_details(feed_url, podcast_id):
    
    resp = requests.get(feed_url)
    bs = BeautifulSoup(resp.content, features='xml')
    
    def cleanhtml(raw_html):
        cleaner = re.compile('<.*?>')
        clean_text = re.sub(cleaner, '', raw_html)
        return clean_text
    
    def hhmmss_sec(time_str):
        h, m, s = time_str.split(':')
        return int(h) * 3600 + int(m) * 60 + int(s)
    
    def mmss_sec(time_str):
        m, s = time_str.split(':')
        return int(m) * 60 + int(s)

    items = bs.findAll('item')
    
    show_items = []
    
    for index, item in enumerate(items):
        episode = {}
        episode['podcast_id'] = podcast_id
        episode['title'] = item.title.text if item.title is not None else None
        episode['description'] = cleanhtml(item.description.text) if item.description is not None else None

        try:
            if ":" in item.duration.text:
                dur_time = item.duration.text
                dur_list = item.duration.text.split(':')

                if len(dur_list) == 3:
                    episode['duration'] = hhmmss_sec(dur_time)

                if len(dur_list) == 2:
                    episode['duration'] = mmss_sec(dur_time)

            else:
                episode['duration'] = item.duration if item.duration is not None else None
        except:
            print("podcast_id = ",podcast_id, " feed_url = ", feed_url, " row number = ", index, " Duration issue")
            pass
        episode['pubDate'] = item.pubDate.text if item.pubDate is not None else None      
        show_items.append(episode)
    
    df_cols = ["podcast_id", "title", "description", "duration", "pubDate"]
    
    df = pd.DataFrame(show_items, columns = df_cols)

    return df

In [ ]:
def concat_all_shows(feeds):
    df_list = []

    for index, rows in tqdm(feeds.iterrows()):
        feed_url = rows["FeedURL"]
        podcast_id = rows["id"]
        
        try:
            df = get_details(feed_url, podcast_id)
            df_list.append(df)
        
        except:
            print(podcast_id, feed_url, "URL Error")
            pass

    final_output = pd.concat(df_list)
    
    return final_output

In [ ]:
df = concat_all_shows(feeds)

In [ ]:
final_results = df

In [ ]:
## Remove brackets from duration
final_results['duration'] = final_results['duration'].astype(str).map(lambda x: x.lstrip('[').rstrip(']'))
final_results.sample(10)

In [ ]:
final_results['duration'] = final_results['duration'].astype(str).map(lambda x: x.lstrip('<itunes:duration>').rstrip('</itunes:duration>'))

In [ ]:
import dateutil
from dateutil.relativedelta import *
from dateutil.easter import *
from dateutil.rrule import *
from dateutil.parser import *

def clean_pubdate(pubdate):
    try:
        date = dateutil.parser.parse(pubdate)
        date = str(date).split(" ")
        date = date[0]
    except:
        print('error', pubdate)
        date = pubdate
        pass

    return date

In [ ]:
final_results['cleanDate'] = final_results['pubDate'].map(lambda x: clean_pubdate(x))

In [ ]:
final_results.replace(',','', regex=True, inplace=True)

In [ ]:
final_results['title'] = final_results['title'].str.lower()
final_results['description'] = final_results['description'].str.lower()

In [ ]:
final_results.replace('\n','', regex=True, inplace=True)

In [ ]:
final_results.replace('\r',' ', regex=True, inplace=True)
final_results.replace('  ',' ', regex=True, inplace=True)

In [ ]:
## remove all links with http or https as well as any emails xx@xxx
final_results.replace('http\S+|www.\S+|\S*@\S*\s?', '', regex=True, inplace=True)

In [ ]:
# remove all hyphens that is not in between words or numbers
# only for the column title and description, because our dates have hyphens
final_results['title'] = final_results['title'].str.replace('-(?!\w)|(?<!\w)-',' ')
final_results['description'] = final_results['description'].str.replace('-(?!\w)|(?<!\w)-',' ')

In [ ]:
# remove all puncuations
# only for the column title and description
import string

def remove_punc(x):
    punc = '''!()[]{};:|"\, <>./?@#$%^&*_~•'''
    try:
        for ele in x:
            if ele in punc:
                x = x.replace(ele, " ")
        cleaned = x
    except:
        cleaned = x
    return cleaned

def remove_punc2(x):
    try:
        cleaned = x.strip(string.punctuation)
    except:
        cleaned = x
    return cleaned

final_results['title'] = final_results['title'].map(lambda x: remove_punc(x))
final_results['description'] = final_results['description'].map(lambda x: remove_punc(x))

final_results['title'] = final_results['title'].map(lambda x: remove_punc2(x))
final_results['description'] = final_results['description'].map(lambda x: remove_punc2(x))

In [ ]:
final_results['title'] = '"' + final_results['title'] + '"'
final_results['description'] = '"' + final_results['description'] + '"'

In [ ]:
cur = con.cursor()

In [ ]:
for eachepisode in final_results.itertuples():
    podcastid  =eachepisode[1] 
    title = eachepisode[2] 
    desc = eachepisode[3]
    dura = eachepisode[4]
    cur.execute('''INSERT INTO episode_detail values (?,?,?,?)''',(podcastid, title, desc, dura ))

cur.close()
con.close()